In [13]:

import pandas as pd
import sqlite3
from datetime import datetime

def get_date_df(date):
    date_str = date.strftime("%Y-%m-%d")
    con = sqlite3.connect('../data/db.sqlite3')
    date_df = pd.read_sql_query('SELECT * from sample_tb WHERE \''+ date_str + ' 00:00\' < ttn_received_at AND ttn_received_at < \'' + date_str + ' 23:59\'', con)
    con.close()
    return date_df

def get_today_df():
    date = datetime.now()
    date_str = date.strftime("%Y-%m-%d")
    con = sqlite3.connect('../data/db.sqlite3')
    today_df = pd.read_sql_query('SELECT * from sample_tb WHERE \''+ date_str + ' 00:00\' < ttn_received_at AND ttn_received_at < \'' + date_str + ' 23:59\'', con)
    con.close()
    return today_df

date = datetime(2023, 1, 23)
date_df = get_date_df(date)

date_df = date_df[['ttn_device_id', 'ttn_received_at', 'ttn_payload_temp', 'ttn_payload_rh', 'ttn_payload_pm1_0', 'ttn_payload_pm2_5', 'ttn_payload_pm10_0']]
date_df

,ttn_device_id,ttn_received_at,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0
0,eui-70b3d57ed0059066,2023-01-23 01:49:01.717729,31.70,60.90,2.0,3.0,3.0
1,eui-70b3d57ed0059066,2023-01-23 02:04:01.758256,31.00,63.79,2.0,4.0,4.0
2,eui-70b3d57ed0059066,2023-01-23 02:19:01.786385,31.29,62.40,0.0,1.0,2.0
3,eui-70b3d57ed0059066,2023-01-23 01:49:01.717729,31.70,60.90,2.0,3.0,3.0
4,eui-70b3d57ed0059066,2023-01-23 02:04:01.758256,31.00,63.79,2.0,4.0,4.0
...,...,...,...,...,...,...,...
87,eui-70b3d57ed0059066,2023-01-23 22:48:50.088917,29.29,64.19,5.0,6.0,6.0
88,eui-70b3d57ed0059066,2023-01-23 23:03:50.122912,29.29,64.09,4.0,6.0,6.0
89,eui-70b3d57ed0059066,2023-01-23 23:18:50.146435,29.29,64.00,4.0,6.0,8.0
90,eui-70b3d57ed0059066,2023-01-23 23:33:50.197491,29.29,64.09,4.0,6.0,9.0


In [14]:

def apply_func(ttn_received_at):
	date = datetime.fromisoformat(str(ttn_received_at))	
	return date.replace(microsecond=0).replace(second=0).replace(minute=0).replace(hour=0)
date_df['ttn_received_at'] = date_df.apply(lambda x: apply_func(x['ttn_received_at']), axis=1)

date_df = date_df.groupby(['ttn_device_id', 'ttn_received_at']).agg(
    ttn_payload_temp = ('ttn_payload_temp', 'mean'),
    ttn_payload_rh = ('ttn_payload_rh', 'mean'),
    ttn_payload_pm1_0 = ('ttn_payload_pm1_0', 'mean'),
    ttn_payload_pm2_5 = ('ttn_payload_pm2_5', 'mean'),
    ttn_payload_pm10_0 = ('ttn_payload_pm10_0', 'mean')
).reset_index()

date_df= date_df.round(decimals=2)

date_df


,ttn_device_id,ttn_received_at,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0
0,eui-70b3d57ed0059066,2023-01-23,28.65,67.56,4.21,6.63,7.26


In [18]:

def apply_func(ttn_received_at):
	return ttn_received_at.strftime('%Y-%m-%d')
date_df['ttn_received_at'] = date_df.apply(lambda x: apply_func(x['ttn_received_at']), axis=1)
date_df


,ttn_device_id,ttn_received_at,ttn_payload_temp,ttn_payload_rh,ttn_payload_pm1_0,ttn_payload_pm2_5,ttn_payload_pm10_0
0,eui-70b3d57ed0059066,2023-01-23,28.65,67.56,4.21,6.63,7.26


In [16]:
date_dicts = date_df.to_dict('records')
date_df.to_dict('records')

[{'ttn_device_id': 'eui-70b3d57ed0059066',
  'ttn_received_at': Timestamp('2023-01-23 00:00:00'),
  'ttn_payload_temp': 28.65,
  'ttn_payload_rh': 67.56,
  'ttn_payload_pm1_0': 4.21,
  'ttn_payload_pm2_5': 6.63,
  'ttn_payload_pm10_0': 7.26}]